In [4]:
import cv2
import numpy as np
import glob
import math
from scipy.spatial import distance
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [15]:
def IrisLocalization(images):
    #convert image to a color image
    target = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    boundary=[] #initialize empty list that will eventually contain all the images with boundaries
    centers=[] #initialize empty list that will contain the centers of the boundary circles
    for img in target:
        
        draw_img=img
        
        # remove noise by blurring the image
        blur = cv2.bilateralFilter(img, 9,75,75)
        img=blur
        
        #estimate the center of pupil
        horizontalProjection = np.mean(img,0);
        verticalProjection = np.mean(img,1);
        center_x=horizontalProjection.argmin()
        center_y=verticalProjection.argmin()
        
        #recalculate of pupil by concentrating on a 120X120 area
        centrecrop_x = img[center_x-60:center_x+60]
        centrecrop_y = img[center_y-60:center_y+60]
        horizontalProjection = np.mean(centrecrop_y,0);
        verticalProjection = np.mean(centrecrop_x,0);
        crop_center_x=horizontalProjection.argmin()
        crop_center_y=verticalProjection.argmin()

        cimg=img.copy()
        cv2.circle(cimg,(crop_center_x,crop_center_y),1,(255,0,0),2)

        #apply Canny edge detector on the masked image
        maskimage = cv2.inRange(img, 0, 70)
        output = cv2.bitwise_and(img, maskimage)
        edged = cv2.Canny(output, 100, 220)
        
        # Apply Hough transform to find potential boundaries of pupil
        circles = cv2.HoughCircles(edged, cv2.HOUGH_GRADIENT, 10, 100)
        
        #define the center of the pupil
        a = (crop_center_x,crop_center_y)
        
        out = img.copy()
        min_dst=math.inf
        for i in circles[0]:
            #find the circle whose center is closest to the approx center found above
            b=(i[0],i[1])
            dst = distance.euclidean(a, b)
            if dst<min_dst:
                min_dst=dst
                k=i
                
        #draw the inner boundary
        print(k[0],k[1],k[2])
        cv2.circle(draw_img, (int(k[0]), int(k[1])), int(k[2]), (255, 0, 0), 3)

        pupil=circles[0][0]
        radius_pupil = int(k[2])
        
        #draw the outer boundary, which is approximately found to be at a distance 53 from the inner boundary 
        cv2.circle(draw_img, (int(k[0]), int(k[1])), int(radius_pupil+53), (255, 0, 0), 3)
        boundary.append(draw_img)
        centers.append([k[0],k[1],k[2]])
    return boundary,centers


In [16]:
file1="D:/College/BE project/Dataset/MMU-Iris-Database/1/left/aeval2.bmp"
#reading the training images from the CASIA dataset
images_train = [cv2.imread(file1)] # for file in sorted(glob.glob('CASIA Iris Image Database (version 1.0)/*/1/*.bmp'))]

#running Localization, Normalization,Enhancement and Feature Extraction on all the training images
boundary_train,centers_train=IrisLocalization(images_train)

215.0 165.0 90.0


In [6]:
import numpy
l=[1,2,3]
l=numpy.array(l)
l=l.reshape(-1,1)
print(l)

[[1]
 [2]
 [3]]


In [7]:
x=1
y=2
print(x,y)

1 2


In [8]:
m=[[3.0673277934209207, 1.7717531512813034, 2.9008398165397686, 1.5938398242673422, 2.421330386700367, 1.7550999393037299, 2.8020361227644126, 1.80001076262922, 2.105282689829557, 1.6286984299447123, 2.5874199992645712, 1.682976039245664, 2.031261219843584, 1.5690871287047208, 2.664524177514414, 1.6570490495232093, 2.020847758497292, 1.6121080620142605, 2.6941792149653336, 1.6207747222103928, 2.0404594088379637, 1.676300569745726, 2.713566950126764, 1.678982532746824, 2.172195286848784, 1.7161438816397414, 2.721594084286987, 1.7549304855259815, 2.133634200842618, 1.6536232490220475, 2.6002347630492166, 1.7204446240045104, 2.35270497157011, 1.6223457057905846, 2.7826891243508687, 1.757149526408992, 2.3118833235921903, 1.768733607699923, 2.79490859789278, 1.7847317339621547, 2.188894639776895, 1.7256558963651756, 2.849775716225486, 1.519157939028593, 2.5401675622075865, 1.8417484990657649, 2.7963998856403496, 1.9536318514790452, 2.103374986927206, 1.6077573765549584, 2.7770398498919606, 1.596294821913085, 2.0437764013890867, 1.538161363546379, 2.6468828086012905, 1.6188076430936542, 2.278332609709344, 1.645194167984828, 2.8139925313531133, 1.5580976253683771, 2.0532280918378536, 1.6360618383764614, 2.657902525184012, 1.6550747005951916, 2.428649266709809, 1.6895249314024579, 2.848962180193775, 1.7043919309312534, 2.6324184332822615, 1.7322924507685813, 2.8177742901431415, 1.801210951577342, 2.4144332116808433, 1.686939284105704, 2.7474396370566403, 1.7150823060493656, 2.3555960704273646, 1.5929307354056212, 2.801418481354908, 1.6507389979240459, 2.342977776886066, 1.5535282759062592, 2.6476343486295693, 1.6943408082927993, 2.3532321633281184, 1.68250444354049, 2.634738583655178, 1.4678721274842024, 2.6971595687191154, 1.8640394769167639, 2.7312228850516966, 1.9552656178289345, 2.7640045729038336, 1.8306620840948353, 2.879889142813981, 1.7876277003064152, 2.572925320088262, 1.8410828806726038, 2.7871090878689304, 1.8697268887211989, 3.5987041382712315, 2.307794882121404, 3.347170334915239, 2.2295790662397335, 3.114071321184074, 2.203239227584488, 3.082554434138411, 2.0515721454356504, 3.0616765984466063, 1.9505760888480639, 3.1056323311313117, 1.7041858985835456, 3.097306650250583, 2.0931237113033534, 2.9536374293638685, 1.9622696820987793, 2.7652069683094287, 1.9252963223540525, 2.6829177205653556, 1.7443059779136187, 2.8607968779846717, 1.8756893267987333, 2.8327834923845723, 1.7915496838122824, 2.9626591213431577, 1.6172890002735105, 3.030851907575072, 1.542573369592945, 2.7338842235157856, 1.537078314012442, 2.8326388437587275, 1.4915477911649466, 2.6244808484374693, 1.606596121961395, 2.6368032223296143, 1.5898764410502135, 3.0268029174457842, 2.106585146363382, 2.7722454215993473, 1.9525494588693837, 3.5291200655875015, 2.266193321186515, 3.2131021827304505, 2.1413477994205032, 4.083405795753947, 2.74002135816862, 3.5744098493237133, 2.35436050989411, 2.7049230005376215, 2.0303948143102186, 2.4738318592388557, 1.8639177671805034, 3.1399043308948986, 2.2771328066897105, 2.7981517416307042, 2.089487305498165, 2.8820854134538814, 2.206201417331333, 2.7371097447941413, 2.172178287759689, 2.8529171173732704, 1.6623061272113, 2.805096943988227, 1.631438818242859, 3.591742596382957, 2.4247314512781255, 3.166484812489047, 2.174703674938291, 2.1458487583093824, 1.6446826261132421, 2.3792305723798814, 1.718883211187537, 2.3745700235929093, 1.8173819128905633, 2.3100989107517975, 1.84816174287866, 2.467546727996659, 1.921918219793981, 2.389508296503516, 1.8364875276578334, 2.519908202704165, 1.9521045156246688, 2.4525311628961757, 1.8466858111703588, 2.372199326428432, 1.641150401775671, 2.520542687422968, 1.7782799674366536, 2.1227691892153624, 1.4520566142271034, 2.5075890035567854, 1.6395595490880313, 2.170248055831453, 1.5311501250127777, 2.5177902067672244, 1.6338456340640235, 2.3212801604734112, 1.7311727464623106, 2.539477321813749, 1.7474221468259223, 2.5469668140336204, 1.8380690670920403, 2.645090014542913, 1.8435439231518318, 2.3308058181350657, 1.8072892061110672, 2.497394332936553, 1.8397988697973968, 2.457704719546606, 1.5475383937903648, 2.643769358739949, 1.6079926783354344, 2.445080300510572, 1.417354851930458, 2.6292561449700163, 1.4663084557660078, 3.3012759270328393, 2.0079757862988625, 3.1511551706077903, 2.0085100959208297, 2.2916963292566996, 1.4532178885334146, 2.5111294638062565, 1.6882949047307219, 2.300927242040459, 1.599826043179013, 2.521675618877567, 1.6987750061425018, 2.2725990563987093, 1.333419483871675, 2.606131228120308, 1.5844717544446336, 2.2534740630334387, 1.575072501053505, 2.6906886082444545, 1.5828955027186749, 2.6244453227975506, 1.754508120392515, 2.769513357078589, 1.6184762506576194, 2.0554322167962313, 1.357190526104568, 2.5506667530245206, 1.5769178892355313, 1.9656915264352153, 1.4725773630050654, 2.419867740976155, 1.690129019908703, 2.0982908224535755, 1.5112180030320554, 2.6733037942597093, 1.6300530325174654, 2.462726466826328, 1.466138373714081, 2.389760117292596, 1.4913999514662226, 1.3582320741175604, 0.879198064159176, 1.4581776397143318, 0.8490657600311323, 1.4864560023948923, 0.7895555112415871, 2.0005264703101338, 0.8413753725910231, 1.2069200842875207, 0.6387085433877058, 1.9067026798150308, 0.8020341096143067, 1.5641074517545723, 0.8611643871482084, 2.341503207116779, 0.8902600810805856, 1.2018431737795117, 0.49411589701938274, 2.195451739572114, 0.43749685903605295, 1.6218867827159413, 0.8383188308021244, 2.339036919731841, 0.8377965397963236, 1.303237837842706, 0.5805762041335144, 2.1805221379756237, 0.6317750515776523, 1.2335092819795506, 0.7066170292547422, 1.9349698096304826, 0.7966524780890664, 1.0568991911801655, 0.640068449589595, 1.6666944471680416, 0.6745656944494217, 1.294958436293895, 0.8203307351186535, 1.721527534269081, 0.7913919363116011, 1.6898060642172696, 1.0613701334606114, 2.149418843336591, 1.129069821046655, 1.4632345133882447, 0.7367043321996423, 2.163335640158669, 0.8354642861602689, 1.541614330556595, 0.6866497367619171, 2.156647854639934, 0.794443502977475, 1.6477577258572345, 0.9414151761377654, 2.2086370059169993, 1.023996763102553, 1.3326704689645918, 0.7798559605775219, 1.9365357647838302, 0.813243886047379, 1.388123976358178, 0.8673888954823743, 2.023247138782681, 0.9569847568860581, 1.5710703296714803, 0.5795879818978149, 2.434628496074226, 0.5215862823434472, 1.6669821492054135, 0.7403288879309602, 2.505917592597006, 0.7163827235330542, 1.0950463135108857, 0.588570399752903, 1.8750007732184963, 0.6428800076890953, 1.4187849934105052, 0.7021943814166798, 2.1706632367573553, 0.7266850613171275, 1.3254973590503476, 0.7802531534053805, 1.9778869468991964, 0.8599298874745545, 1.3570910928019373, 0.7502602403529044, 1.9673688570630898, 0.8663971944487208, 1.5752685861506017, 0.7178901466780663, 2.453494106811907, 0.6572097885258747, 1.2090886280382198, 0.7763430698415233, 1.8310029962969177, 0.8034339268970863, 1.3719490892973623, 0.6549552284253959, 2.01384601735991, 0.6784926059900654, 1.0734274512574031, 0.6222185270423364, 1.6498126308585004, 0.6473012722705664, 1.2199902162838754, 0.8017166780037357, 1.612628281673007, 0.8516519617844301, 1.3004566000407434, 0.8216176942329677, 1.5369026282239329, 0.8336373853100689, 1.248839701575323, 0.7490595652752827, 1.4940113691751438, 0.8195331600197774, 1.4126258794096, 0.8113020137687654, 1.8689634121886955, 0.9326844371869094, 1.7104761578296226, 0.7905381235016481, 2.258832363612663, 0.7113010664127104, 1.33498732650831, 0.8668976309283574, 2.0107773384800076, 0.8159685498228917, 1.3532747866610424, 0.7683800919848944, 1.564407117713834, 0.8630979005325012, 1.4940382592300292, 0.7509059073194153, 1.9299513278245652, 0.7537792229183031, 1.2686789661236624, 0.6162492172293276, 1.818889976125265, 0.7564647034980081, 1.3962617459005668, 0.7146251882197805, 1.8934798222958658, 0.902501904722316, 1.212551644023158, 0.66952700817909, 1.7318898522899007, 0.7680386655471074, 0.7874942130497636, 0.40354875759561526, 1.2066347794992547, 0.5220848691726372, 0.7145986838761385, 0.41364247144069843, 0.9316872651611391, 0.594418573988716, 1.836663160673897, 0.9979260679683901, 1.7150527896825611, 0.8687292906114316, 1.5542322242508997, 0.9436466575809357, 1.3745972597726923, 0.7989490018315148, 1.6935879272090935, 1.267584052812261, 1.6355589103230024, 1.1679509167132796, 1.8804120744081116, 1.1548358036389386, 2.0075000509269088, 0.9452297931865418, 1.4110798079381404, 0.7674558759202827, 1.5860604648076178, 0.9445521145762942, 0.8178296137796173, 0.5871089804236544, 1.066981892246462, 0.6892357308656956, 0.9293473155929292, 0.5904949261635198, 0.9494598872573679, 0.6210364537913098, 1.9085382815995642, 1.0948174564418856, 1.9457817205205503, 1.064002582490059, 1.6246184139390953, 0.9993080533529476, 2.1608867546405177, 1.0437365041096924, 1.2107381796757577, 0.7126005682095125, 1.3562383468536, 0.7616048202599599, 0.9848629010689902, 0.6736293923780384, 0.9436209781335476, 0.656057398322872, 1.3484793945074809, 0.9940111939091463, 1.2811687374355796, 0.9233074839717325, 1.4904440965388321, 1.0824945122421121, 1.3466394199020775, 0.9027684600728696, 1.4568032130328277, 0.9862239780119515, 1.2414872553367802, 0.8655487880710246, 1.1665541759942868, 0.7984489019529424, 1.079139700250648, 0.6747743525981013, 0.6954411435035688, 0.3586955937244087, 0.887877249210608, 0.32441668434139803, 0.6493409951255069, 0.34187948591951334, 0.907057838250773, 0.3631562720796512, 0.7797142585986521, 0.44754816790971363, 1.0735522124780346, 0.4739444367905513, 0.8019737464665211, 0.47733449756377255, 0.9969531388443218, 0.5257157885524608, 1.0228184102707685, 0.4834796597398625, 1.3844445489908186, 0.4717275611825387, 1.092989313625826, 0.697817925293627, 1.4793216738153674, 0.6667532260413953, 1.1067081994871293, 0.5330205547733156, 1.706475719032077, 0.48521162711194166, 1.0385376822247039, 0.6723625714638239, 1.6511840942217293, 0.67750854324573, 1.0958909660462113, 0.5840343620595825, 1.6318120131695883, 0.6219956474646886, 1.5245891591940004, 0.8805888783984195, 1.5440470903545171, 0.7419180777272204, 1.4655931034834535, 0.9830769093133531, 1.642658227954145, 0.8412266660724881, 1.1130023696973814, 0.5872347846300093, 1.6774494945832652, 0.6644310516928629, 1.2606073281066317, 0.669042591754831, 2.0248795235877024, 0.6688963481904114, 1.2487915595927919, 0.5900445897349398, 2.2337400000357137, 0.529847322820269, 1.3399759394951323, 0.5254314795735214, 2.404546465448562, 0.4453629913303643, 1.2601036627820692, 0.7075384501629287, 2.064200261064382, 0.7758524156164966, 1.524981081596683, 0.6455482174346747, 2.4606088946934257, 0.552020190312602, 1.186097905822893, 0.3255091949123714, 2.3010106781185264, 0.3136865483250073, 1.4085659833977977, 0.6643612867664554, 2.3257879122411653, 0.6140520095510951, 1.460446988299739, 0.8515448016483311, 2.423517549742563, 0.7693001678543987, 1.296619946610888, 0.4786749714416392, 2.357330150186135, 0.4516535609816949, 1.3771668345134103, 0.31877867401733223, 2.527844484135986, 0.2659242897024775, 1.3705867312661058, 0.27723698378338474, 2.5094329213093385, 0.23361146980887015, 1.568617186984198, 0.4189661597102049, 2.6510463091902583, 0.349650083741437, 1.4365984762486126, 0.270571823915677, 2.59707263583614, 0.2307507843796237, 1.3273390617450458, 0.5332104478343378, 2.407624175338922, 0.4777402008245117, 1.2547177937910858, 0.4830652136449621, 2.289792443566453, 0.4325832455303967, 1.3518099350217376, 0.35735329188932397, 2.411901844419925, 0.31867402667735756, 1.2738081600813955, 0.5042556786330714, 2.302207463584024, 0.44026851042137327, 1.2537976435560152, 0.4839227602718202, 2.274328021656162, 0.4221522566956585, 1.2217063712760854, 0.5619111153870587, 2.2806564607434265, 0.4825117872712776, 1.2872037803401568, 0.43576474332915005, 2.2737709121020853, 0.4469385870861736, 1.321832744993276, 0.5558117751954216, 2.3828283143816424, 0.47255257887977203, 1.359950552319101, 0.43369290530939364, 2.3994248882705684, 0.37330068494513635, 1.2401472377697726, 0.3583202559765352, 2.3142136471581716, 0.3168991373754502, 1.4431320090050817, 0.617597786432722, 2.338749671462889, 0.57517022777319, 1.1476813062809013, 0.5952583072226663, 2.0941448095489896, 0.5759623131959308, 1.2150017726176994, 0.7080252954491333, 2.0539055423991277, 0.6728494172436674, 1.4808714601158466, 0.7672370048707788, 2.1545540088797455, 0.7625159711042339, 1.4220141646408164, 0.6636240637074386, 2.226234641396477, 0.6872641258708851, 1.300987160829492, 0.5238965307844208, 2.3287262770033177, 0.46111613685228775, 1.2600319938984872, 0.4565832241288267, 2.2760729736719623, 0.4408664441010164, 1.139293983772179, 0.685455572171727, 1.9941263893985157, 0.6188456078836612, 1.3087642942122315, 0.738388861677733, 2.038158868331861, 0.755937639884174, 1.2380119215898748, 0.6210728429082791, 2.05082821071221, 0.6678848569927586, 1.3829764932673045, 0.7418732577136988, 2.207448517704853, 0.6708956937237522, 1.1425094036410202, 0.6935137119894185, 2.052086998198172, 0.618637893052634, 1.3916517170886962, 0.6289508415216878, 2.2664289998917386, 0.5745731116584116, 1.2642022178612498, 0.6941610962363267, 2.1553391858776836, 0.6893612144053703, 1.1715655048751383, 0.7091923162779318, 1.376139902719685, 0.7756040854032243, 1.7808084571589058, 1.1202704680778846, 1.5702253375178565, 1.054995536135534, 2.1165730942700036, 1.277353324510053, 1.925083118697081, 1.0140408472067852, 3.013312208200602, 1.7015479331751981, 2.6194451123406695, 1.4474936201070407, 1.5911043767410593, 0.9005371324882908, 1.6525081418619592, 0.919874730413574, 1.2094568564727999, 0.6945880987709984, 1.1034903759622954, 0.6653845318853775, 1.4615402589962887, 1.0456024628037004, 1.3952310507517853, 1.0422501943233575, 2.848400490909265, 1.6933550922186202, 2.648544269466617, 1.4481308309084353, 2.6689094515365475, 2.037773968372562, 2.497231626436239, 1.7655548744528302, 2.011607204031592, 1.353843016658625, 1.7567584069475655, 1.1143509079479843, 0.9722431096544286, 0.829690341981894, 0.9230922974646334, 0.7956646260511513, 1.644638422954048, 0.9793866615237989, 1.3998149639262296, 0.8657452724760528, 1.2777462832628579, 1.0846128836925182, 1.1785408097311618, 0.9945988380985837, 1.3075793764866974, 1.01767214880239, 1.138160906187262, 0.8909385449656106, 0.3482276175525661, 0.21401868967270476, 0.4550244219859884, 0.25201663210803366, 0.3568701894069635, 0.19296057074693773, 0.45056454464124784, 0.2396694383272906, 0.44796621801847036, 0.22306335214483908, 0.48374254881629386, 0.2531314786388428, 0.6509528274796363, 0.4084845558609317, 0.5694632778394014, 0.3438496637192927, 0.6342499876197307, 0.3786801860052351, 0.5452866175749742, 0.3460221187866419, 0.4330532517968855, 0.22288902457007814, 0.36497318169028403, 0.22209420322608886, 0.5626564336922036, 0.40533977416502165, 0.47701282327068045, 0.36351356134474283, 0.8210510698760051, 0.5113819354217315, 0.6723430663077072, 0.461682995783315, 0.9496945843363467, 0.5744816764649792, 0.936597545121811, 0.5204193971893616, 1.0692338673390356, 0.5608133239062708, 1.1495352925623772, 0.5847255273298452, 1.1590936326763948, 0.8878496130155842, 1.0763501351980223, 0.6834548216556797, 1.1033868825045985, 0.6821826119760076, 1.0726419036989734, 0.6328999309296486, 0.9101013186484939, 0.6077557773447376, 1.187272289131271, 0.5962056719721833, 0.9602148389181313, 0.5743299892539626, 1.4612474958679655, 0.6143179095993514, 1.0577845625421995, 0.7246011480093787, 1.7295032542806936, 0.7192580526889134, 1.2704797313031793, 0.6689782367215884, 2.059860370865395, 0.6809158933734049, 1.0899303675844907, 0.5055242538294287, 1.8356265724239194, 0.5906622416985068, 1.195597597469491, 0.7335780436515698, 1.8036724116594058, 0.7803117216655207, 1.2814768426839365, 0.7405818508052303, 2.0144109222894913, 0.6975737420710886, 1.4424974585052421, 0.735539554485036, 2.1816168772647258, 0.7701562504383919, 1.1870467463105094, 0.6601125325058766, 2.1009104765810753, 0.6659533789335638, 1.3513865157506948, 0.4735958818784403, 2.395145821412788, 0.41435780499614827, 1.405024552288657, 0.44065143022693515, 2.516238402222573, 0.36206262165648895, 1.4465115636856218, 0.431679977797091, 2.580817799436982, 0.3579221344777189, 1.4072101566743962, 0.19192187701931435, 2.5610422199219487, 0.16584023147453483, 1.4454036442454525, 0.30307762113156145, 2.571195330174983, 0.2528358446496666, 1.3629268005637356, 0.6251852143228853, 2.404916363914161, 0.5543663183285714, 1.41973069619698, 0.5398333451306614, 2.3343293761319703, 0.5745359131864409, 1.3455012913249063, 0.5532685172269096, 2.369588918244048, 0.44590402015293296, 1.095635510801984, 0.5928262632687913, 2.0047663221855583, 0.6342061671680999, 1.5520688484106366, 0.8433931342446802, 2.3935198327210774, 0.820718980129612, 1.2757933189478994, 0.36686934529628656, 2.382306626941298, 0.3205541441302123, 1.2454021891974048, 0.42164361129354005, 2.262417321255057, 0.3738847030311918, 1.3951662215988836, 0.750964450352408, 2.30490528449491, 0.7267838889514799, 1.224592997929271, 0.6572613427810254, 2.1503688326250523, 0.578589625777067, 1.2212632593654376, 0.7252022074433517, 1.9979892080306865, 0.6623302309172958, 1.2432321196739995, 0.6662139904453842, 2.102825672264264, 0.5928152145489327, 1.271934367915915, 0.6070206635694019, 2.1289484451636573, 0.605959536635235, 1.1273524385071008, 0.7155813232783483, 1.7659541878861393, 0.7593764360594706, 1.1252857987606548, 0.5797590214075306, 1.7811709639296125, 0.6150746770038014, 0.9663712033002367, 0.5076256826720947, 1.763916418555239, 0.48185593515096503, 1.203784835079501, 0.6485145023653636, 1.8933176080541543, 0.6826069574828106, 1.1101872038849134, 0.645634182885108, 1.8950602051155543, 0.6060786861031507, 1.1364933340893655, 0.5766539611500427, 1.944194917422701, 0.5692902896548246, 1.3332851388819766, 0.6266471536598015, 2.1326510083816874, 0.598430372742882, 1.0792423251148808, 0.5764803831087332, 1.805491278661579, 0.5872858625415329, 1.11595905739222, 0.5457433345725433, 1.8347023279934505, 0.49855592842247176, 0.9986058054654405, 0.5122345567001672, 1.7419760195176281, 0.5020815487239453, 1.085246349573358, 0.5870686188055192, 1.7633821756070456, 0.5754964367109472, 1.1235849336138326, 0.6521790629271836, 1.6176847591433794, 0.7459037824569985, 1.0941540243539187, 0.5971881074288692, 1.233287306374611, 0.7148247554848874, 1.5522544464071464, 0.9386428472123863, 1.6129388922975325, 0.927659340478602, 1.0638508509132676, 0.6451950021124304, 1.0576830849077856, 0.6046417919275526, 0.7527425532940382, 0.4558336331620857, 0.7047559970412755, 0.4143436905328313, 1.5347324990850506, 1.0882083222994927, 1.4366107010348126, 0.9951042310251994, 2.10386597042462, 1.2625710516370177, 2.0253672590793173, 1.1623557108954272, 2.181729890717236, 1.3350318204778624, 2.3555620226962586, 1.262237336913948, 3.040416571364614, 1.9331095788472252, 2.990371800473345, 1.8253650679709756, 2.1430216820922645, 1.1390935335214745, 2.575597548011705, 0.7919662072454202, 2.3820460211893537, 1.6406447387470369, 2.2487160228693144, 1.5698423615469714, 2.3766163304818555, 1.5966261323404385, 2.4179428513490935, 1.5325472287156305, 2.15218057592387, 1.3412116469797186, 2.291226126918832, 1.4108867999954549, 1.9747116245719425, 1.2945588389303908, 2.263976538534744, 1.1713258667639745, 1.4177582250622693, 0.8762616641649608, 1.4925091514842865, 0.8272499753581538, 0.9061217295106676, 0.8009859469335262, 0.8405278653172055, 0.7503841290819145, 0.5107620486037932, 0.4809030886396705, 0.47138035927799915, 0.4056331244688025, 0.2636933752384726, 0.1935488053326335, 0.28667658067075075, 0.174661769629308, 0.5637131314961685, 0.3598670276938912, 0.5458791311517024, 0.34451620850181985, 0.8330118720467474, 0.505313812545432, 0.7717323420442439, 0.46156970174566375, 0.6944516229131718, 0.4283809726788849, 0.7287911417325688, 0.4471226926803399, 0.8227339335797664, 0.4999813255210058, 0.9025411158492128, 0.48349746478731603, 0.7508145100010073, 0.6154338525303996, 0.9339829519220998, 0.5393469459949606, 0.8659129042896494, 0.5856534385175802, 0.9163785247162004, 0.4636211780198992, 0.5155652500986453, 0.339909872834793, 0.6641245674291629, 0.3324531201296981, 0.6503183359645737, 0.4467840634524547, 0.6253380011448175, 0.39566916332422164, 0.8024616169148832, 0.5226945739937282, 0.9262441315335802, 0.41811368422282813, 0.532873807689813, 0.3000251332448524, 0.8472430317120235, 0.3148616485144502, 0.7383127608335642, 0.4067144172250955, 0.9975712358717786, 0.38694491999659536, 0.9957482029057867, 0.5046457429506823, 1.3250519189575827, 0.6153335744644745, 1.062831264617837, 0.5612619798498635, 1.4873867335162236, 0.6395103711405097, 0.9734409803327595, 0.5539440568680624, 1.5043029392213945, 0.6092537032908982, 0.9938983952901819, 0.6135360718039677, 1.4329434204385532, 0.6661852938459234, 1.0941008349411208, 0.6864551668013843, 1.482787660479476, 0.7372864262750423, 1.3409487794350805, 0.6612476785242662, 1.8160796384356483, 0.7143999446183192, 1.1739081248249046, 0.6251677387983015, 1.878777304271301, 0.6813580962320971, 1.2485571898498327, 0.604444997778147, 2.0585699276547156, 0.6162024324697876, 1.1388149726941847, 0.49935168318500245, 2.087028977227238, 0.4684973318281664, 1.2817726933153162, 0.5073033895325086, 2.3192117472331013, 0.43790815755086676, 1.264125851059427, 0.3412063821144891, 2.341657120807077, 0.3045080184412683, 1.2283614222224697, 0.47732276525714845, 2.2432985799131187, 0.39673876832823796, 1.2217712831592782, 0.6759514444947949, 2.1302886093935998, 0.6236793769793555, 1.2560607705353715, 0.7497139246780011, 2.045692072561991, 0.7526821991658558, 1.2917604530732285, 0.6785712538247144, 2.1617847248555924, 0.6340076231686462, 1.3731344771332794, 0.7090120777472659, 2.1661124531125977, 0.7023398381243336, 1.0291323418487823, 0.5775720427670973, 1.7872830893083922, 0.6316627820334146, 1.4496471536869726, 0.7963505644082174, 2.2477277507397595, 0.7400574115005812, 1.0629386330558213, 0.6307437408680625, 1.8753735550168618, 0.6257487490930075, 1.333013432305905, 0.8146958478101725, 2.111630795074566, 0.7622394231697612, 1.0260793388682372, 0.5994808977609292, 1.747169111205871, 0.645724249264017, 1.1709622308687204, 0.6941826029876144, 1.617292871043316, 0.6901403151414691, 1.1124151062194807, 0.6517844209619261, 1.6917003866494302, 0.649629204168866, 1.167047215427122, 0.5056631717719547, 1.965976218120625, 0.5024062480385973, 1.1702841096929382, 0.6517250201702874, 1.9922463183965713, 0.5843815212072009, 1.081342784364574, 0.5494799064312815, 1.7498468428299623, 0.5775151225278974, 1.105451897355843, 0.6435949556245357, 1.5952197766468008, 0.6308497940345279, 1.0326104764753126, 0.6061632057703132, 1.531540740158515, 0.6360011916886864, 1.0066167905994095, 0.6000212578834317, 1.5799885053690632, 0.5994032986440604, 1.195249412689944, 0.707121750987504, 1.777749977944175, 0.6249893400610911, 0.9992892604578065, 0.560675226665815, 1.5929795247823086, 0.614714141135932, 1.238949249648827, 0.6450515685842557, 1.6736783705169072, 0.7393671383396148, 1.1377863285390992, 0.5817181749887361, 1.69801876852135, 0.6415473546553725, 0.9830364165041072, 0.5352348774981222, 1.433449629677702, 0.6325450717624912, 0.9989529179719374, 0.5953888426787008, 1.4147770122347532, 0.648649606854238, 0.8392573989292459, 0.45754781978491676, 1.2567629759123444, 0.5334714861183898, 0.6653251511066962, 0.38578852074273506, 0.8728033267293303, 0.42779962738917965, 0.4450287164671645, 0.2683590265544791, 0.5800121755024317, 0.3280418602599454, 0.9353453654389127, 0.6867522617440891, 1.0028933921813745, 0.670929185826709, 2.116120628288227, 1.5075661368135016, 2.0861430518869684, 1.5320960691420915, 2.3151508261243503, 1.408585230764456, 2.3599453336710137, 1.3692692874992618, 2.671633134361151, 1.491202189017737, 2.7944577320601565, 1.3239909359938915, 3.0845720163107835, 1.8930707785812526, 3.063319315682193, 1.8515447063939374, 2.3419485435994494, 1.4088239718682052, 2.5915695174133866, 1.5170621381389533, 1.8714225059009193, 1.002331855798972, 2.7942109734675817, 0.753765722085231, 2.6023773017315888, 1.3666730370474873, 2.7751528837997075, 1.5405769658807933, 2.2391368054783847, 1.4316762740062865, 2.610693653042982, 1.5722571900457758, 1.5591491256757977, 0.5627710320924889, 2.520582061730512, 0.6729663754671773, 1.7515180192790412, 1.1339243913294, 2.126011796379891, 1.1126617817250575, 1.918653766229294, 1.2278337451271275, 2.119049692298973, 1.1875044131967563, 2.4598014722637025, 1.2169086989322686, 2.4323630285179623, 1.1504491991863501, 1.8420066130162014, 1.050983889814149, 1.8031885068276836, 0.995844895650101, 1.235260231984708, 0.9165730873378726, 1.1035805384997688, 0.8679425944441571, 0.4276826982003852, 0.28230519580013647, 0.49539837617034227, 0.323098708780415, 0.31740648454821535, 0.17053821928381668, 0.400814794895964, 0.20519854269541823, 0.4077620365853236, 0.216856960189324, 0.5324645681187627, 0.25623289326840987, 0.43725118221882064, 0.21967921544385854, 0.5331228870306586, 0.2369128619870911, 0.41333731082032354, 0.2263983816528344, 0.5304653130779938, 0.25397885226806693, 0.46631701770084816, 0.23771201944910111, 0.5399659133903719, 0.2944536687058299, 0.49646974243739395, 0.25689756105427075, 0.6953036207124174, 0.2895825575860034, 0.5859665382407372, 0.42636760754607733, 0.6350328268748598, 0.34054396201160364, 0.6423380062889318, 0.3774637331485625, 0.6887455887126138, 0.34772938646325224, 0.6794911952471504, 0.4351016255300765, 0.9184536691291881, 0.40122866778294, 1.0595861304363619, 0.5584225591013401, 1.1654859432735862, 0.6022967637644328, 1.1273895241169698, 0.6841806179826954, 1.2127407122709235, 0.6420467592721635, 0.9817966857807695, 0.5978358558838797, 1.2135914729629744, 0.569101107026567, 0.9998809544365096, 0.5620574266401663, 1.307162563603836, 0.5861315492634149, 1.0710831351854897, 0.7172771367350127, 1.3816550533046383, 0.6987828357065212, 1.0737617014734613, 0.7235485149838173, 1.4659612009538159, 0.6500367264967508, 1.4545807760974783, 0.9945680520398565, 1.7161109749517516, 0.9024525055013771, 1.6246479915641303, 0.9549178355041312, 1.9533095237932787, 0.9398633686519193, 1.4205159878913478, 0.9756993470562746, 2.0105913946805303, 0.9371101272316628, 1.388699119535231, 0.8487440111210272, 2.0972335206464203, 0.7497144382532608, 1.6390782730974982, 0.8779460437993356, 2.060124836988186, 0.8738582880753679, 1.5533697001882831, 1.0365797801711791, 2.2259381237965346, 0.8926149788462783, 1.3548229316657023, 0.6974229787876683, 2.057716687302777, 0.6229413873625873, 1.3178184196194462, 0.7294598549924132, 1.7254336515297686, 0.7978298400025059, 1.3536881973690496, 0.796760573486057, 1.79446776240384, 0.8232421620504218, 1.1579664876282687, 0.7216247531006004, 1.6527043134990314, 0.7136685159148777, 1.4751761154930394, 0.8049377289832764, 1.990174075834196, 0.7753533470282868, 1.3508285305476315, 0.8720345175272688, 1.8439751865396798, 0.8611147083547851, 1.4160783009860336, 0.6893723536832812, 1.8127837734776198, 0.653859063430131, 1.050916054114639, 0.6231259408560931, 1.4955814331872594, 0.6807418324398851, 1.3328335596799752, 0.7867397600422699, 1.7942461112210484, 0.7256794855183047, 1.1939920362578116, 0.6936616122100625, 1.4689553019396016, 0.7262493995763001, 1.2570734058502988, 0.6630994413861809, 1.5745377537624148, 0.724730404857497, 1.4763325645520706, 0.8179218610469423, 1.74574270745969, 0.8773081815708805, 1.2263375079124659, 0.7738259050550662, 1.7453130832676091, 0.7408490372155044, 1.2662402438944707, 0.882755910559639, 1.8164170516411597, 0.8289694098314326, 1.496252978628824, 1.0105484772533573, 1.8975059713463835, 0.9968738435477336, 1.323073153549041, 0.8272392599361311, 1.677225829702126, 0.7990907983032477, 1.2952671447656687, 0.9185447420648274, 1.734516451216491, 0.8434097754365371, 1.1670286528433909, 0.7308017323359979, 1.5767708309877055, 0.7220759876050116, 1.1055709491145633, 0.6888735698769897, 1.5534069991940962, 0.6419015410935356, 1.014011839712003, 0.7130505851633335, 1.3506750528021303, 0.6518200097675231, 1.1965460883870485, 0.7269785431011282, 1.3433663209961102, 0.6950514443095445, 2.3248609869202626, 1.3167524202315575, 2.1204315544970136, 1.195231350631854, 2.18689209587576, 1.243780560316935, 2.0270585927713203, 1.2191543635551314, 1.6224037419104493, 0.9594204245741003, 1.6224126214338195, 0.8625517979118847, 0.9480931072296399, 0.6736250959574843, 1.0122118688067057, 0.6543970810605145, 0.6039874125277497, 0.3660479075219906, 0.7028295926473086, 0.34379759429964757, 1.405427291128956, 0.8469985383199898, 1.6618293773240926, 0.871013701553232, 2.6277177273044607, 1.7498646331491505, 2.8212785159836256, 1.850728787324425, 2.783314774775641, 1.8537475078822343, 3.1266781499419407, 1.7984848449112478, 1.953648755894958, 1.3770225007916173, 2.1342103142572673, 1.3234797385933117, 3.3292334460148414, 1.856701595652105, 3.466013091122766, 1.8384261384353955, 2.2084931203755187, 1.5146466687463334, 3.1339154838558576, 1.2875519837531713, 1.8659411030526143, 1.0129039068155485, 3.0497222347077315, 0.7862877353515816, 2.0335946202373574, 1.2195728100706418, 2.9769709233099313, 1.034972247954188, 2.416699092031789, 1.7222045152857566, 3.1349437684852894, 1.5923086667094355, 1.9485617999277485, 1.047823291814666, 3.1563365100382472, 0.8023700376375957, 1.7769513082517698, 1.0518159838842305, 2.9514041144116065, 0.8354751436520078, 2.178757684443756, 1.4390094044758754, 2.9445349276538915, 1.352825854972004, 2.1778708135075937, 1.31932667951237, 2.672472522821928, 1.3099130070301874, 2.8078803022177894, 1.6978545153128444, 2.8814610723897225, 1.6912964952926726, 2.3185830285010085, 1.6467726520967738, 2.7464493931551126, 1.5765049759075336, 1.9142039634295087, 1.4770680749676162, 2.3848788604656344, 1.2845956293608856, 1.5918861028443505, 0.9203391155014106, 1.6866783812712611, 1.0411656552081057, 0.6254938856395741, 0.4738012904720139, 0.6835031617483431, 0.519924124065025, 0.2326575153360034, 0.14503226679452036, 0.26347477560131627, 0.1727802479806649, 0.29568621796343453, 0.15685394239011932, 0.33789566750178973, 0.1624068622012244, 0.4607382203783441, 0.2737045987040686, 0.4601072259934842, 0.2684261883103293, 0.5130085778113647, 0.2944608648063455, 0.5249558103500903, 0.2879873591752621, 0.40040528482373133, 0.26158443484356597, 0.4671260146666252, 0.24277574148476075, 0.4229779709706768, 0.2810333760458771, 0.4197533708013864, 0.2563471468441767]]

In [9]:
for i in m:
    print(len(i))

1536


In [6]:
l=[[1,2,2],[2,3,4]]
print(l[0][0])

1


In [8]:
from math import*
 
def euclidean_distance(x,y):
 
    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))
 
print(euclidean_distance([0,3,4,5],[7,6,3,-1]))

9.746794344808963


In [25]:
for i in range(108):
    path="d/bhhc/dbh/"
    no='{0:03d}'.format(i+1)
    p2="/1/*.bmp"
    print(path+no+p2)

d/bhhc/dbh/001/1/*.bmp
d/bhhc/dbh/002/1/*.bmp
d/bhhc/dbh/003/1/*.bmp
d/bhhc/dbh/004/1/*.bmp
d/bhhc/dbh/005/1/*.bmp
d/bhhc/dbh/006/1/*.bmp
d/bhhc/dbh/007/1/*.bmp
d/bhhc/dbh/008/1/*.bmp
d/bhhc/dbh/009/1/*.bmp
d/bhhc/dbh/010/1/*.bmp
d/bhhc/dbh/011/1/*.bmp
d/bhhc/dbh/012/1/*.bmp
d/bhhc/dbh/013/1/*.bmp
d/bhhc/dbh/014/1/*.bmp
d/bhhc/dbh/015/1/*.bmp
d/bhhc/dbh/016/1/*.bmp
d/bhhc/dbh/017/1/*.bmp
d/bhhc/dbh/018/1/*.bmp
d/bhhc/dbh/019/1/*.bmp
d/bhhc/dbh/020/1/*.bmp
d/bhhc/dbh/021/1/*.bmp
d/bhhc/dbh/022/1/*.bmp
d/bhhc/dbh/023/1/*.bmp
d/bhhc/dbh/024/1/*.bmp
d/bhhc/dbh/025/1/*.bmp
d/bhhc/dbh/026/1/*.bmp
d/bhhc/dbh/027/1/*.bmp
d/bhhc/dbh/028/1/*.bmp
d/bhhc/dbh/029/1/*.bmp
d/bhhc/dbh/030/1/*.bmp
d/bhhc/dbh/031/1/*.bmp
d/bhhc/dbh/032/1/*.bmp
d/bhhc/dbh/033/1/*.bmp
d/bhhc/dbh/034/1/*.bmp
d/bhhc/dbh/035/1/*.bmp
d/bhhc/dbh/036/1/*.bmp
d/bhhc/dbh/037/1/*.bmp
d/bhhc/dbh/038/1/*.bmp
d/bhhc/dbh/039/1/*.bmp
d/bhhc/dbh/040/1/*.bmp
d/bhhc/dbh/041/1/*.bmp
d/bhhc/dbh/042/1/*.bmp
d/bhhc/dbh/043/1/*.bmp
d/bhhc/dbh/

In [4]:
for i in range(3):
    for j in range(3):
        path="D:/"
        no1='{0:03d}'.format(i+1)
        no2='{0:03d}'.format(j+1)
        p1="/1/*.bmp"
        p2="/2/*.bmp"
        l=[]
        s1=path+no2+p1
        s2=path+no1+p2
        l.append(s1)
        l.append(s2)
        print(l)
    print()

['D:/001/1/*.bmp', 'D:/001/2/*.bmp']
['D:/002/1/*.bmp', 'D:/001/2/*.bmp']
['D:/003/1/*.bmp', 'D:/001/2/*.bmp']

['D:/001/1/*.bmp', 'D:/002/2/*.bmp']
['D:/002/1/*.bmp', 'D:/002/2/*.bmp']
['D:/003/1/*.bmp', 'D:/002/2/*.bmp']

['D:/001/1/*.bmp', 'D:/003/2/*.bmp']
['D:/002/1/*.bmp', 'D:/003/2/*.bmp']
['D:/003/1/*.bmp', 'D:/003/2/*.bmp']



In [6]:
l=[[1,2,3],[1,3,4],[1,1,1]]
s=[1,1,1,1,1]
print(l.count(1))
print(s.count(1))


0
5


In [7]:
print(l)

[[1, 2, 3], [1, 3, 4], [1, 1, 1]]
